In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)



In [3]:
from implementations import *

In [10]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [11]:
def get_tr_and_te(y, tx, k_indices, k):
    te_y = y[k_indices[k]]
    te_tx = tx[k_indices[k]]
    tr_indices = []
    for i, indices in zip(range(len(k_indices)), k_indices):
        if i != k:
            tr_indices.append(indices)
            
    tr_indices = np.array(tr_indices).flatten()
    return tx[tr_indices], y[tr_indices], te_tx, te_y

In [75]:
y1 = np.array([1,1,-1,-1, 1, 1])
x1 = np.array([1,2,3,4,5,6])
k = 3
k_indices = build_k_indices(y1, k, 23)
print(k_indices)
print(get_tr_and_te(y1, x1, k_indices, 0))

[[5 4]
 [2 1]
 [0 3]]
(array([3, 2, 1, 4]), array([-1,  1,  1, -1]), array([6, 5]), array([1, 1]))


In [73]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return np.exp(t)/(1+np.exp(t))

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    cost = 0.0
    n = len(y)
    for i in range(0, len(y)):
        cost += (np.log(1 + np.exp(tx[i].T@w)) - y[i]*tx[i].T@w)/n
    
    return cost

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T@(sigmoid(tx@w) - y)

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    
    gradient = calculate_gradient(y, tx, w)
    
    w = w - gamma*gradient
    
    return loss, w

def my_logistic_reg(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    losses = []
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return w, loss

In [106]:
def cross_validation_reg_log_regr(y, tx, w_initial, max_iters, gammas, lambdas_, k_fold, seed):
    k_indices = build_k_indices(y, k_fold, seed)
    tr_losses = np.zeros((len(gammas), len(lambdas_)))
    te_losses = np.zeros((len(gammas), len(lambdas_)))

    for gamma_index,gamma in zip(range(len(gammas)), gammas):
        for lambda_index, lambda_ in zip(range(len(lambdas_)),lambdas_):
            tr_k_losses = np.zeros((k_fold))
            te_k_losses = np.zeros((k_fold))
            weights_k = np.zeros((k_fold))
            for k in range(k_fold):
                tr_tx_k, tr_y_k, te_tx_k, te_y_k = get_tr_and_te(y, tx, k_indices, k)
                
                w_k, tr_loss_k = reg_logistic_regression(tr_y_k, tr_tx_k, lambda_, w_initial, max_iters, gamma)
                #w_k, tr_loss_k = my_logistic_reg(tr_y_k, tr_tx_k, w_initial, max_iters, gamma)
                
                te_loss_k = calculate_loss_sigmoid(te_y_k, te_tx_k, w_k)
                
                tr_k_losses[k] = tr_loss_k
                te_k_losses[k] = te_loss_k
                
            tr_loss = np.mean(tr_k_losses)
            te_loss = np.mean(te_k_losses)
            weight = np.mean(weights_k)
            tr_losses[gamma_index][lambda_index] = tr_loss
            te_losses[gamma_index][lambda_index] = te_loss
            
            print(tr_loss)
            print(te_loss)
            argmin = np.argmin(te_losses)
            gamma_idx = argmin//len(lambdas_)
            lambda_idx = argmin%len(lambdas_)
            gamma = gammas[gamma_idx]
            lambda_ = lambdas_[lambda_idx]

    return tr_losses, te_losses, gamma, lambda_

In [127]:
#Global variables
max_iters = 2500
k_fold = 3
seed = 142
lambdas_ = np.array([100, 1000, 10000])
gammas = np.array([10**(-i) for i in range(5,10)])
print("gammas = ", gammas)
w_initial = np.array([0.0 for i in range(tX.shape[1])])
tX_standardized, tr_mean, tr_std = standardize(tX)

#to work with loss
y = np.array([int((y[i] + 1)/2) for i in range(len(y))])

gammas =  [1.e-05 1.e-06 1.e-07 1.e-08 1.e-09]


In [128]:
tr_losses, te_losses, gamma, lambda_ = cross_validation_reg_log_regr(y, tX_standardized, w_initial, max_iters, gammas, lambdas_, k_fold, seed)

At iteration 0, loss = 0.6931471805599424
At iteration 1000, loss = 1.121579901871062
At iteration 2000, loss = 2.2034012683175748
At iteration 0, loss = 1.119733951912261
At iteration 1000, loss = 2.170528846816451
At iteration 2000, loss = 3.5900982935283174
At iteration 0, loss = 2.164408863360957
At iteration 1000, loss = 3.59228204399471
At iteration 2000, loss = 1.1180637840816787
2.303868309007598
188329.42507864253
At iteration 0, loss = 3.899812979794199
At iteration 1000, loss = 1.363073988541969
At iteration 2000, loss = 2.282972025802477
At iteration 0, loss = 1.3590409028983332
At iteration 1000, loss = 2.2545735519186496
At iteration 2000, loss = 4.00493039952471
At iteration 0, loss = 2.2484722559278416
At iteration 1000, loss = 4.01149020909986
At iteration 2000, loss = 1.3535656551665716
2.55122919993676
206391.57946863945
At iteration 0, loss = 4.88609634014767
At iteration 1000, loss = 1.4834045476067004
At iteration 2000, loss = 3.3010983438979813
At iteration 0, lo

In [63]:
#tX_standardized, tr_mean, tr_std = standardize(tX)
#w, loss = least_squares(y,tX)

In [131]:
print("gamma = ", gamma, " lambda = ", lambda_)
w, loss = reg_logistic_regression(y, tX_standardized, lambda_, w_initial, 8000, gamma)

gamma =  1e-06  lambda =  100
At iteration 0, loss = 0.5570391434507874
At iteration 1000, loss = 0.55624586649447
At iteration 2000, loss = 0.5558813685185672
At iteration 3000, loss = 0.555709338725023
At iteration 4000, loss = 0.5556260152222255
At iteration 5000, loss = 0.5555845888231217
At iteration 6000, loss = 0.5555634390152187


In [112]:
DATA_TEST_PATH = 'data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_standardized = (tX_test- tr_mean)/tr_std

In [132]:
OUTPUT_PATH = 'data/output.csv' 
y_pred = predict_labels(w, tX_test_standardized)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)